In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Treinamento dos modelos

Neste arquivo são feitos os treinamentos e testes dos modelos. A sequência para a obtenção dos resultados é dada por:

- Etapa 1: prepara os dados
- Etapa 2:divide os dados entre dados de treinamento de dados de teste. 
- Etapa 3: se desejado, aplica uma transformação nos dados.     
- Etapa 4: treina o modelo com os dados de treinamento.
- Etapa 5: testa o modelo nos dados de teste e calcula a métrica.
- Etapa 6: salva os nomes dos modelos, o nome das transformações e o valor obtido para a métrica em um arquivo.

In [2]:
import pandas as pd
import numpy as np

# transfomações
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import Normalizer

# para divisao dos dados
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

# modelos
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

# para importar exporttar arquivos
from pathlib import Path
import os

# para não mostrar mensagens com warnings
import warnings

import time

In [13]:
# importa os dados
current_path = os.getcwd()
path_dados=Path(current_path)/"dados"
df=pd.read_csv(path_dados/"dados_semOutliers.csv")
df.head(3)

,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class
0,27.0,M,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,C
1,25.0,M,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,A
2,31.0,M,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,C


In [14]:
#Etapa 1: Mapeia a variável 'gender' para valores inteiros
gender_map = {'M': 0, 'F': 1}
var = 'gender'
df[var] = df[var].map(gender_map)

In [15]:
# Etapa 2: 
# seleciona variaveis de treinamento e alvo e divide 
# os dados

var_train=[var for var in df.columns if var != 'class'] 

X = df[var_train]   
y = df['class'] 

# divide os dados
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size=0.1,
                                                    random_state=0)

# verificação
print("Dimensão dos dados de treinamento: ",X_train.shape)
print("Dimensão dos dados de teste: ",X_test.shape)


Dimensão dos dados de treinamento:  (11370, 11)
Dimensão dos dados de teste:  (1264, 11)


## Modelos com as transformações

In [16]:
# lista com trasformações
transformacoes=[('sem_transformacao', None),
                ('yeo_johnson',PowerTransformer(method="yeo-johnson")),
                ('quantile',QuantileTransformer(output_distribution="normal")),
                ('standard', StandardScaler()),                
                 ]

In [17]:
# dicionário com os classificadores
classificadores={'decisionTree':
                 DecisionTreeClassifier(
                     criterion="entropy",
                     max_depth=10,
                     min_samples_leaf=5,
                     min_samples_split=18,
                     min_impurity_decrease=0,
                     random_state=0
                 ),
                
                'randomForest':
                 RandomForestClassifier(
                     max_depth=14,
                     max_features=8,
                     n_estimators=80,
                     bootstrap=True,
                     max_samples=None,
                     random_state=0
                 ),
                 
                 'kNearestNeighbour':
                 KNeighborsClassifier(
                     n_neighbors=50,
                     weights='distance'
                 ),
                                                          
                 'rbf_SVM':
                 svm.SVC(kernel="rbf",
                         random_state=0,
                         gamma=2.9,
                         max_iter=1000,
                         C=0.1),
                                                          
                 'poly3_SVM':
                 svm.SVC(kernel="poly",
                         degree=5,
                         random_state=0,
                         gamma=2.9,
                         max_iter=5000,
                         C=0.5),                
                }
                

In [18]:
#colunas que serão transformadas
listaVar=['age', 'height_cm', 'weight_kg',
 'body fat_%', 'diastolic', 'systolic',
 'gripForce', 'sit and bend forward_cm',
 'sit-ups counts', 'broad jump_cm']

In [19]:
warnings.filterwarnings("ignore")

dict_metricas={}

for trans in transformacoes:
    
    print(f"Treinamento para dados com a transformação: {trans[0]}")
    t_ini=time.time()
    
    # copia os dados
    X_train_temp = X_train.copy()
    X_test_temp = X_test.copy()             
        
    
    #Etapa 3: aplica as transformações
    
    if trans[1]!=None:
        
        #seleciona dados para transfromação
        dados_train = X_train_temp[listaVar].values  
        dados_test = X_test_temp[listaVar].values
        
        # fita a transformação com dados de treinamento
        obj = trans[1].fit(dados_train)  
        
        # transforma os dados de teste e treinamento
        X_train_temp[listaVar] = obj.transform(dados_train)
        X_test_temp[listaVar] = obj.transform(dados_test)


    
    # Etapas 4 e 5: treina e testa o modelo
    
    metricas_modelo=[]
    
    for nome_modelo, clf in classificadores.items():    
        
        # treina, testa o modelo
        clf.fit(X_train_temp,y_train)        
        y_hat = clf.predict(X_test_temp)       
        print(f'-> {nome_modelo}: treinado!')
        
        # armazena a métrica
        metricas_modelo.append(metrics.accuracy_score(y_test, y_hat))         
         
               
    # armazena os resultados
    dict_metricas[trans[0]]=metricas_modelo  
              
    print(f"tempo: {time.time()-t_ini}seg.\n") 

    

Treinamento para dados com a transformação: sem_transformacao
-> decisionTree: treinado!
-> randomForest: treinado!
-> kNearestNeighbour: treinado!
-> rbf_SVM: treinado!
-> poly3_SVM: treinado!
tempo: 10.652427673339844seg.

Treinamento para dados com a transformação: yeo_johnson
-> decisionTree: treinado!
-> randomForest: treinado!
-> kNearestNeighbour: treinado!
-> rbf_SVM: treinado!
-> poly3_SVM: treinado!
tempo: 13.794661045074463seg.

Treinamento para dados com a transformação: quantile
-> decisionTree: treinado!
-> randomForest: treinado!
-> kNearestNeighbour: treinado!
-> rbf_SVM: treinado!
-> poly3_SVM: treinado!
tempo: 11.92281699180603seg.

Treinamento para dados com a transformação: standard
-> decisionTree: treinado!
-> randomForest: treinado!
-> kNearestNeighbour: treinado!
-> rbf_SVM: treinado!
-> poly3_SVM: treinado!
tempo: 13.228384971618652seg.



In [20]:
# apresentação dos resultados
df=pd.DataFrame(dict_metricas)
df.index=list(classificadores.keys())
df

,sem_transformacao,yeo_johnson,quantile,standard
decisionTree,0.690665,0.689873,0.689873,0.690665
randomForest,0.738924,0.738133,0.742089,0.740506
kNearestNeighbour,0.565665,0.621044,0.606804,0.605222
rbf_SVM,0.465981,0.516614,0.511867,0.485759
poly3_SVM,0.330696,0.352057,0.287975,0.371044


In [21]:
# Etapa 6: salva os resultados em formato adequado
df=df.reset_index(drop=False).rename(columns={'index':'modelos'})
df.to_csv(path_dados/'metricas_modelos.csv', index=False)

## Análise dos dados gerados

A análise dos dados gerados foi feita no notebook "1_analise completa_dos_dados.ipynb" na seção 8.2, "Análise dos resultados dos modelos".